In [1]:
import pandas as pd
import numpy as np
import cv2
from skimage import measure
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline

In [2]:
sub = pd.read_csv('../result/lgb_filter_olddata_deep_15.csv')
sub.set_index('ImageId', inplace=True)
sub.fillna('',inplace=True)
noship = sub.loc[sub.EncodedPixels.str.len() == 0]
ships = sub.loc[sub.EncodedPixels.str.len() > 0]
len(ships)

4264

In [3]:
def get_mask(img_id, df):
    shape = (768,768)
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    masks = df.loc[img_id]['EncodedPixels']
    if(type(masks) == float): return img.reshape(shape)
    if(type(masks) == str): masks = [masks]
    for mask in masks:
        s = mask.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1
    return img.reshape(shape).T

def get_mask_single(df, idx):
    shape = (768,768)
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    mask = df.iloc[idx,0]
    if(type(mask) == float): return img.reshape(shape)
    
    s = mask.split()
    for i in range(len(s)//2):
        start = int(s[2*i]) - 1
        length = int(s[2*i+1])
        img[start:start+length] = 1
    return img.reshape(shape).T

In [4]:
def get_rectangle(mask, plot = True, shape = (768, 768)):

    ret,thresh = cv2.threshold(mask,0,1,0)
    img, contours, hier = cv2.findContours(thresh, 1, 2)
    cnt = max(contours, key=len)
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)

#     draw = np.stack((thresh*255,)*3, axis=-1).astype(np.uint8)
    empty = np.zeros(shape, dtype=np.uint8)
    # res = cv2.drawContours(draw,[box],0,(0,0,255),-1)
    res = cv2.drawContours(empty,[box],0,1,-1)
    if plot:
        fig,axis = plt.subplots(1,2)
        axis[0].imshow(mask)
        axis[1].imshow(res)
#         print(mask.shape)
    
    return res

def decode_mask(mask, shape=(768, 768)):
    pixels = mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [5]:
def rectanglize(df, idx, area_threshold = 3000, plot=False):
    elr = df.iloc[idx,0]
    #print(elr)
    mask = get_mask_single(ships, idx)
    area = mask.sum()
    if area > area_threshold:
        res = get_rectangle(mask, plot=plot)
        res = decode_mask(res)
    else:
        res = elr
    return res, area
    

In [6]:
areas = []
ship_dict = []
for idx in tqdm_notebook(range(len(ships))):
    rect, area = rectanglize(ships, idx, 3000, plot=False)
#     print(rect)
    ship_dict.append({'ImageId':ships.index[idx], 'EncodedPixels': rect})
    areas.append(area)

In [7]:
rect_ships = pd.DataFrame(ship_dict)

In [8]:
to_sub = pd.concat([rect_ships, noship.reset_index()])

/home/kownse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [9]:
import kaggle_util
kaggle_util.save_result(to_sub, '../result/resunet_rectangle_up3000.csv', 
                        competition = 'airbus-ship-detection', 
                        send = True, index = False)

save result
upload result
cmd: kaggle competitions submit -c airbus-ship-detection -f ../result/resunet_rectangle_up3000.csv.7z -m "submit"


In [10]:
to_sub.head()

,EncodedPixels,ImageId
0,311233 2 311998 7 312766 2 312769 4 313533 9 3...,000367c13.jpg
1,176890 2 177658 4 178425 7 179193 9 179960 12 ...,0010551d9.jpg
2,403198 2 403963 5 404729 8 405494 11 406259 14...,002a943bf.jpg
3,441689 3 442455 7 443222 9 443989 10 444757 11...,0035268d9.jpg
4,534890 3 535658 3 536426 4 537193 5 537962 4 5...,0035268d9.jpg
